In [1]:
import os
import sys

from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import udf


from snowflake.snowpark import Session
import pandas as pd
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)



/usr/local/lib/python3.10/dist-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1.dev0+ga6eabc2b.d20230609), please install a version that adheres to: 'pyarrow; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
# Unpack env vars
API_BASE = "http://localhost:8000/v1/completions"
SNOW_ROLE = os.environ["SNOW_ROLE"]
SNOW_WAREHOUSE = os.environ["SNOW_WAREHOUSE"]
SNOW_DATABASE = os.environ["SNOW_DATABASE"]
SNOW_SCHEMA = os.environ["SNOW_SCHEMA"]
SNOW_TABLE = "MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO" #os.environ["SNOW_TABLE"]
SNOW_COLUMN = 'ID' # os.environ['SNOW_COLUMN']

table_sample = '' #int(os.environ.get('table_sample'))

temperature = float(os.environ['temperature'])
max_tokens = int(os.environ['max_tokens'])

# Establish snowpark session
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': SNOW_ROLE,
    'warehouse': SNOW_WAREHOUSE,
    'database': SNOW_DATABASE,
    'schema': SNOW_SCHEMA,
     'session_parameters':{
            'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
        }
}


session = Session.builder.configs(connection_params).create()
token = get_token()



In [3]:
@udf(name="build_prompts", is_permanent=True, stage_location="@files", replace=True, session=session)
def build_prompt(name: str, feature_1: str, feature_2: str, target_customer1: str,feature_3: str) -> str:
    PROMPT_TEMPLATE = '''Here is a catchy product description for a {0} that highlights {1} and {2}. Personalize the description for {3} interested in {4}:'''
    return PROMPT_TEMPLATE.format(name, feature_1, feature_2, target_customer1,feature_3)

def get_df(table_name, n = None):
    df = session.table(table_name)
    df_new= df.select(df["id"],df["name"], build_prompt(df["name"], df["feature_1"], df["feature_2"], df["target_customer1"], df["feature_3"]).as_("prompt"))
    return df_new.select("id", "name", "prompt")


In [4]:
df_s = get_df("MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO",
            table_sample).collect()
df = pd.DataFrame(df_s)
df.head()

,ID,NAME,PROMPT
0,A1234,EJOffice Standup Desk,Here is a catchy product description for a EJO...
1,A3214,EJOffice Chair,Here is a catchy product description for a EJO...
2,A3215,EJOffice Regular Desk,Here is a catchy product description for a EJO...
3,A3216,EJOffice Trash Wastebasket,Here is a catchy product description for a EJO...
4,A3217,EJOffice Paper Shredder,Here is a catchy product description for a EJO...


In [6]:
prompt_list=df["PROMPT"].values.tolist()

outputs = [client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.2",
                                      prompt=p, max_tokens=200, temperature=0.5) for p in prompt_list]

descriptions = []
# Print the outputs.
for output in outputs:
        generated_text = output.choices[0].text
        descriptions.append(generated_text.strip())
        print(f"Generated text: {generated_text!r}")
df['DESCRIPTION'] = descriptions

Generated text: '\n\n🌱🏠 Upgrade your home office to a healthier, more productive workspace with the EJOffice Standup Desk! 🏋️📈\n\nOur ergonomic standup desk, crafted from sustainable, solid wood, is the perfect solution for remote workers seeking to reduce back pain, improve posture, and boost energy levels. 🌿\n\n💡 Save money and reduce clutter by replacing your bulky, expensive traditional desk with our affordable, adjustable standup desk. 💰\n\n🌱🌱 Made of eco-friendly, sustainably sourced wood, our standup desk is not only good for your health but also kind to the planet. 🌍\n\n🏋️🏋️ Easily switch between sitting and standing positions throughout the day with a simple twist of the knob, promoting'
Generated text: "\n\nIntroducing the EJ Leather Executive Office Chair, designed specifically for remote workers seeking optimal comfort and support while working long hours from home. This ergonomic marvel boasts a sleek, modern design that will elevate your home office setup, all while provi

In [7]:
DESTINATION_TABLE = "PRODUCT_DESCRIPTIONS"
DESTINATION_COLUMNS = 'ID,NAME,DESCRIPTION'
 
_ = session.write_pandas(df[DESTINATION_COLUMNS.split(',')], 
                           DESTINATION_TABLE, 
                           auto_create_table=True, 
                           overwrite=True,
                           on_error='continue')